In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import mariadb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential

In [2]:
conn = mariadb.connect(
    user="user",
    password="password",
    host="host",
    port=port,
    database="database"
)

#DB메인코드
# 커서로 sql문 실행
cursor = conn.cursor()
#실행할 SQL문
query = "SELECT CONVERT(content USING UTF8),disdata from voicedata"
cursor.execute(query)
result = cursor.fetchall()
 # 커서닫기
cursor.close()
conn.close()

In [3]:
X = [i[0] for i in result]
y = np.array([i[1] for i in result]).astype('float64')
df = pd.DataFrame({"document": X, "label": y})

In [4]:
MAX_LEN = 1000
TRUNC = "pre"

train_input, val_input, train_target, val_target = train_test_split(df["document"], df["label"], test_size=0.4, stratify=df["label"])
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_input)
train_sequences = tokenizer.texts_to_sequences(train_input)

train_seq = pad_sequences(train_sequences, maxlen=MAX_LEN, truncating=TRUNC)
val_sequences = tokenizer.texts_to_sequences(val_input)
val_seq = pad_sequences(val_sequences, maxlen=MAX_LEN, truncating=TRUNC)

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=100000, output_dim=64, input_length=MAX_LEN))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.(2, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.MLP(2, return_sequences=False)))
model.add(tf.keras.layers.Dropout(rate=0.3))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'MLP'

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_seq, train_target, epochs=50, batch_size=32, validation_data=(val_seq, val_target))